# Исследование надёжности заёмщиков

На основании статистики о платежеспособности клиентов,полученной от кредитного отдела банка, требуется определить влияние семейного положения, количесва детей, уровня дохода и цели получения кредита на факт погашения кредита в срок для построения модели **кредитного скоринга**.

**Цель исследования** — проверить четыре гипотезы:

- Влияет ли количество детей на возврат кредита в срок;
- Влияет ли семейное положение на возврат кредита в срок;
- Влияет ли размер дохода на возврат кредита в срок;
- Влияет ли цель получения кредита на возврат кредита в срок.
    
**Ход исследования** -

Информация о заемщиках получена из файла заказчика `/datasets/data.csv`. О качестве данных ничего не известно, поэтому перед началом проверки гипотез следует провести обзор данных на предмет ошибок и их влияния на результат исследования.
Затем, на этапе предобработки данных необходимо скорректировать ошибки, которые являются критическими, выполнить категоризацию данных и ответить на поставленные вопросы.

**Описание данных:**

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

    
Этапы исследования:
1. [Изучение общей инфорации](#general_id)
2. [Предобработка данных](#preprocessing_id)
- 2.1.[Обработка пропусков](#preprocessing_one_id)
- 2.2.[Замена типа данных](#preprocessing_two_id)
- 2.3.[Удаление дубликатов](#preprocessing_three_id)
- 2.4.[Лемматизация](#preprocessing_four_id)
- 2.5.[Категоризация данных](#preprocessing_five_id)
3. [Ответы на вопросы](#answers_id)
4. [Общий вывод](#conclusion_id)
5. [Чек-лист готовности проекта](#list_id)

## Изучение общей инфорации
<a id="general_id"></a>

In [1]:
#импорт библиотеки pandas
import pandas as pd
# импортируем pymystem3:
from pymystem3 import Mystem





In [2]:
#чтение файла с данными
data = pd.read_csv('/datasets/data.csv')

In [3]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


В названиях колонок нарушений не выявлено.

Есть следущие замечания к данным из исходной таблицы:
* `days_employed` - наличие отрицательного трудового стажа;
* `education` - строчные буквы сочетаются с прописными;
* `purpose` - цели не приведены к единообразию


Получаем информацию о таблице data:


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак,таблица с входными данными от банка содержит 12 столбцов и 3 типа данных.

Количество значений в столбцах общий трудовой стаж в днях и ежемесячный доход отличаются по сравнению с другими столбцами.
Значит,в данных есть пропущенные значения.


**Вывод**

В строках таблицы представлены входные данные о Клиентах банка. Часть колонок описывает личную информацию о Клиенте банка, такую как: 
пол,возраст,образование,количество детей,семейное положение,данные о трудовом стаже. А часть колонок содержит информацию о платежеспособности Клиента и цель получения кредита.

Предварительно можно утверждать,что, данных для проверки гипотез достаточно.
Но есть пропуски в сведениях о трудовом стаже и ежемесячном доходе,которые требуют внимания.

Также,важно выявить природу появления отрицательных значений в информации об общем трудовом стаже,привести к единообразцию колонку с целью получения кредита и информации об образовании. 


## Предобработка данных
<a id="preprocessing_id"></a>

### Обработка пропусков
<a id="preprocessing_one_id"></a>


In [5]:
#выполним подсчет пропусков в таблице data
data.isna().sum()


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Как и предполагалось ранее, пропуски обнаружены в колонках с данными о трудовом стаже и доходе сотрудников.
Пропущено 2174 значения, что представляет собой около 10% от всей таблицы.

Предположим, что для определенных типов занятости возможно не заполнять информацию о трудовом стаже и доходе.

In [6]:
# проверим взаимосвязь типа занятости и пропусках в информации о трудовом стаже и доходе
# путем группировки данных по столбцу "тип занятости" и выбрав нулевые значения
link_isna_income_type = data[data['days_employed'].isna()].groupby('income_type')['days_employed','income_type','total_income'].count()
print(link_isna_income_type.head(10))

                 days_employed  income_type  total_income
income_type                                              
госслужащий                  0          147             0
компаньон                    0          508             0
пенсионер                    0          413             0
предприниматель              0            1             0
сотрудник                    0         1105             0


/tmp/ipykernel_123/1584623610.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  link_isna_income_type = data[data['days_employed'].isna()].groupby('income_type')['days_employed','income_type','total_income'].count()


Возможно и не требуется заполнять трудовой стаж для наших сотрудников и пенсионеров?
Сотрудников смогут согласовать их руководители,а пенсионеры уже заработали свой стаж.
Сразу встает вопрос,а сколько всего типов занятости у нас есть?


In [7]:
#выведем количество заявок на согласование кредита по типу занятости
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [8]:
# напишем цикл, который найдет медиану по столбцу общий трудовой стаж в днях и общий доход
# по каждому типу занятости и заполнит пропущенные значения

#создадим таблицу с перечнем типов занятости 
income_type_unique = data['income_type'].unique()

# зададим цикл for для элементов таблицы с типами занятости
for i in income_type_unique:
        #отфильтруем таблицу data по типу заянтости и выведем столбец с трудовым стажем
        days_employed_type = data.loc[data['income_type'] == i,'days_employed']
        # найдем медиану по столбцу трудовой стаж
        m = days_employed_type.median()
        # заменим все нулевые значения по столбцу трудовой стаж на значение медианы
        data['days_employed'] = data['days_employed'].fillna(m)
        # отфильтруем таблицу data по типу заянтости и выведем столбец с общим доходом
        total_income_type = data.loc[data['income_type'] == i,'total_income']
        # найдем медиану по столбцу общий доход
        n = total_income_type.median()
        # заменим все нулевые значения по столбцу общий доход на значение медианы
        data['total_income'] = data['total_income'].fillna(n)

#выведем результаты медиан на печать
        print("Значение медианы по колонке стаж:",m)
        print("Значение медианы по колонке доход:",n)
#выведем результаты работы цикла на печать
print(data.isna().sum())


  

Значение медианы по колонке стаж: -1574.2028211070854
Значение медианы по колонке доход: 142594.39684740017
Значение медианы по колонке стаж: 360505.66854353686
Значение медианы по колонке доход: 128747.67556966442
Значение медианы по колонке стаж: -1574.2028211070854
Значение медианы по колонке доход: 162401.35155456822
Значение медианы по колонке стаж: -2385.358043088401
Значение медианы по колонке доход: 142594.39684740017
Значение медианы по колонке стаж: 366413.65274420456
Значение медианы по колонке доход: 131339.7516762103
Значение медианы по колонке стаж: -1047.5254523012309
Значение медианы по колонке доход: 320878.77089724294
Значение медианы по колонке стаж: -578.7515535382181
Значение медианы по колонке доход: 98201.62531401133
Значение медианы по колонке стаж: -3296.7599620220594
Значение медианы по колонке доход: 53829.13072905995
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_stat

In [9]:

#так как по колонке стаж часть значениий медианы меньше 0,заменим их на 0

for el in data['days_employed']:
    if el < 0:
        data['days_employed'] = data['days_employed'].replace(el,0)
#выведем результаты работы цикла на печать   
 
display(data.head(10))
        

        


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,0.000000,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,0.000000,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,0.000000,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,0.000000,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,0.000000,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,0.000000,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,0.000000,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,0.000000,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,0.000000,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Исходя из общего количества заявок,видно,что данных о стаже и доходе не хватает примерно около 10% по каждому типу занятости. Предположим,что информацию о сотрудниках получают при связи с линейным руководителем сотрудника для согласования.
Стаж по пенсионерам нам не важен,но неплохо было бы видеть доход.
Таким образом, у нас не заполнено около 10% данных о стаже и доходе, несмотря на это,вернемся к цели нашего исследования.
Цель - выявить зависимость между количеством детей,семейным положением и фактом погашения кредита в срок,
поэтому если мы выполним замену пропущенных значений на значения медианы, наши данные не пострадают.
Так как речь идет о таких показателях как сумма дохода и данные об общем трудовом стаже замена данных на 0 даст еще большую погрешность,если бы таких пропусков у нас было около 1%, отклонения были бы несущественны и можно смело менять их на 0.
Но значение медианы по колонке общий трудовой стаж меньше 0, поэтому в этом случае мы вынуждены заменить их на 0.



### Замена типа данных
<a id="preprocessing_two_id"></a>

In [10]:

# заменим вещественный тип данных на числовой методом astype заменив исходный столбец
data['days_employed'] = data['days_employed'].astype('int64')
data['total_income'] = data['total_income'].astype('int64')

# проверим,что получилось
data.info(memory_usage='deep')

#Так как данные разных типов хранятся раздельно, мы исследуем использование памяти разными типами данных.
#Начнём со среднего показателя использования памяти по разным типам данных.
for dtype in ['float','int','object']:
    selected_dtype = data.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 11.7 MB
Average memory usage for float columns: 0.00 MB
Average memory usage for int columns: 0.14 MB
Average memory usage for object columns: 1.76 MB


In [11]:
# заменим вещественный тип данных на числовой методом astype заменив исходный столбец
for dtype in ['int']:
    selected_dtype = data.select_dtypes(include=[dtype])
    selected_dtype_column = selected_dtype.columns
    for i in selected_dtype_column:
        data[i] = data[i].astype('int32')


#заменим object на category

for dtype in ['object']:
    selected_dtype = data.select_dtypes(include=[dtype])
    selected_dtype_column = selected_dtype.columns
    for i in selected_dtype_column:
        data[i] = data[i].astype('category')
print(data.info(memory_usage='deep'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21525 non-null  int32   
 1   days_employed     21525 non-null  int32   
 2   dob_years         21525 non-null  int32   
 3   education         21525 non-null  category
 4   education_id      21525 non-null  int32   
 5   family_status     21525 non-null  category
 6   family_status_id  21525 non-null  int32   
 7   gender            21525 non-null  category
 8   income_type       21525 non-null  category
 9   debt              21525 non-null  int32   
 10  total_income      21525 non-null  int32   
 11  purpose           21525 non-null  category
dtypes: category(5), int32(7)
memory usage: 705.9 KB
None


**Вывод**
Для того,чтобы сделать правильные выводы требуется увидеть закономерность.
Заметить ее значительно легче при отсутствии лишних сведений.
Стаж измеряется в днях,а сумму дохода можно округлить до целого числа.




### Обработка дубликатов
<a id="preprocessing_three_id"></a>

In [12]:
#в столбце education приведем строки к нижнему регистру
data['education'] = data['education'].str.lower()

#проверим таблицу на наличие дубликатов
data.duplicated().sum()

# удалим дубликаты
data = data.drop_duplicates().reset_index(drop = True)

#проверим таблицу на наличие дубликатов
data.duplicated().sum()
                

0

**Вывод**
    Перед обаботкой дубликатов данные были приведены к единообразию путем приведения информации об образовании к нижнему регистру. Дубликаты удалены методом data.drop_duplicates() так как у нас нет уникальных занчений в столбцах и мы принимаем за уникальное значение всю строку. При проверке были обнаружены дублирующиеся строки в количестве 71 шт. Возможно при заполнении формы Клиенты отправили заявку дважды.

### Лемматизация
<a id="preprocessing_four_id"></a>

In [13]:
#выведем уникальные значения для столбцов
for i in data.columns:
    unique = data[i].unique()
    print(i, unique)
    



children [ 1  0  3  2 -1  4 20  5]
days_employed [     0 340266 400281 ... 362161 373995 343937]
dob_years [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
education ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
education_id [0 1 2 3 4]
family_status ['женат / замужем', 'гражданский брак', 'вдовец / вдова', 'в разводе', 'Не женат / не замужем']
Categories (5, object): ['женат / замужем', 'гражданский брак', 'вдовец / вдова', 'в разводе', 'Не женат / не замужем']
family_status_id [0 1 2 3 4]
gender ['F', 'M', 'XNA']
Categories (3, object): ['F', 'M', 'XNA']
income_type ['сотрудник', 'пенсионер', 'компаньон', 'госслужащий', 'безработный', 'предприниматель', 'студент', 'в декрете']
Categories (8, object): ['сотрудник', 'пенсионер', 'компаньон', 'госслужащий', 'безработный', 'предприниматель', 'студент', 'в декрете']
debt [0 1]
total_inc

In [14]:
purpose_unique = data['purpose'].unique()

m = Mystem()

# создадим функцию по созданию категории для цели
def create_category_purpose(row):
    lem_purpose = m.lemmatize(row['purpose'])
    if 'автомобиль' in lem_purpose:
        return 'автомобиль'
    if ('жилье' in lem_purpose) or ('недвижимость' in lem_purpose):
        return 'недвижимость'
    if 'образование' in lem_purpose:
        return 'образование'
    if 'свадьба' in lem_purpose:
        return'свадьба'


       

**Вывод**

Применение лемматизации потребовалось только к столбцу цели.
Все цели можно отнести к 4 категориям: недвижимость, приобретение автомобиля, оплата образования, организация свадебного мероприятия.
Для определения категорий была создана функция create_category_purpose.
Данная функция создана с использованием функции lemmatize, которая определеяет ячейку,где указана основа слова наименования категории.



### Категоризация данных
<a id="preprocessing_five_id"></a>

In [15]:
#применим функцию и создадим столбец с категорией по цели
data['purpose_category'] = data.apply(create_category_purpose,axis=1)

#проверим все ли категории присвоены
print(data['purpose_category'].value_counts())
print(data['purpose_category'].count())

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64
21454


In [16]:
# проверим количество клиентов с 20 детьми
print("Число клиентов с 20 детьми:",data[data['children'] == 20]['children'].count())

# заменим отрицательное значение в столбце дети
data['children'] = data['children'].replace(-1,1)


Число клиентов с 20 детьми: 76


In [17]:
#уточним максимальный стаж для наших клиентов (по законодательству РФ минимальный возраст для начала трудовой деятельности 14 лет)
max_days_employed = (data['dob_years'].max()-14)*365

#отфильтруем таблицу по значению стажа, который превышает допустимо возможный стаж
data_type = data[data['days_employed'] >= max_days_employed]

#узнаем по каким типам заложен нереальный стаж
print(data_type['income_type'].unique())

#так как это та группа по которой скорее всего стаж не влияет на задолженность по кредиту,заменим их на 0

for el in data['days_employed']:
    if el >= max_days_employed:
        data['days_employed'] = data['days_employed'].replace(el,0)
#выведем результаты работы цикла на печать   
 
print(data.head(10))

#проверим уникальные значения по возрасту с применением сортировки
data.sort_values(by = 'dob_years')['dob_years'].unique()



['пенсионер', 'безработный']
Categories (2, object): ['пенсионер', 'безработный']
   children  days_employed  dob_years education  education_id  \
0         1              0         42    высшее             0   
1         1              0         36   среднее             1   
2         0              0         33   среднее             1   
3         3              0         32   среднее             1   
4         0              0         53   среднее             1   
5         0              0         27    высшее             0   
6         0              0         43    высшее             0   
7         0              0         50   среднее             1   
8         2              0         35    высшее             0   
9         0              0         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0     

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75], dtype=int32)

In [18]:
# проверим,что же случилось с полом и выведем строки со значением 'XNA' в столбце gender
print(data[data['gender'] == 'XNA'])

       children  days_employed  dob_years            education  education_id  \
10684         0              0         24  неоконченное высшее             2   

          family_status  family_status_id gender income_type  debt  \
10684  гражданский брак                 1    XNA   компаньон     0   

       total_income               purpose purpose_category  
10684        203905  покупка недвижимости     недвижимость  


In [19]:
def category_total_income(row):
    if row <= 50000:
        return 0
    if row <= 100000:
        return 1
    if row <= 150000:
        return 2
    if row <= 200000:
        return 3
    else:
        return 4
    
data['total_income_category'] = data['total_income'].apply(category_total_income)

print(data['total_income_category'].value_counts())

    



2    7807
4    5066
3    4118
1    4091
0     372
Name: total_income_category, dtype: int64


**Вывод**

   Для проведения анализа требуется отнести анализируемые данные к категориям, для того, чтобы можно было определить зависимость изменения анализируемого элемента от категории.
В данном исследовании необходимо определить зависимость между возвратом кредита в срок и:
* `наличием детей`
* `семейным пложением`
* `уровнем дохода`
* `целью получения кредита`

  
Наличие детей и семейное положение не требует доп. категорий, так как их довольно немного.
Для уровня дохода было создано 5 категорий.
* `0 - доход до 50 000 руб.`
* `1 - доход от 50 001 - 100 000 руб.`
* `2 - доход от 100 001 - 150 000 руб.`
* `3 - доход от 150 001 - 200 000 руб. `
* `4 - доход от 200 001 руб. и выше `

На основании созданных категорий можно сделать вывод, что основная масса Клиентов получает кредит на приобретение или ремонт недвижимости, менее на приобретение авто, чуть реже на образование и очень редко на проведение свадьбы.

По доходам большая часть наших Клиентов получает доход в размере от 100-150 тыс. руб., меньше Клиентов с доходом более 200 тыс. руб., и реже обращаются клиенты с доходом от 150 - 200 и 50 - 100 тыс. руб., Клиентов с доходом до 50 тыс. руб. совсем мало.

## Ответьте на вопросы
<a id="answers_id"></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [20]:
#сгруппируем таблицу по количеству детей и выведем информацию по просроченной задолженности
data_groupby_children = data.groupby('children').agg({'debt':'sum','children':'count'})
#выведем информацию о доле задолженности по каждой категории
data_groupby_children['debt_percent'] = data_groupby_children['debt']/data['debt'].sum()*100
#узнаем долю каждой категории 
data_groupby_children['children'] = data_groupby_children['children']/data['children'].count()*100
#определим зависимость количества детей и задолженности
data_groupby_children['bound'] = data_groupby_children['children']/data_groupby_children['debt_percent'] 

#выведем таблицу на экран
display(data_groupby_children.sort_values(by = 'bound',ascending = False))



,debt,children,debt_percent,bound
children,,,,
5,0,0.041950,0.000000,inf
0,1063,65.680060,61.056864,1.075720
3,27,1.538175,1.550833,0.991838
1,445,22.629813,25.560023,0.885360
2,194,9.564650,11.143021,0.858353
4,4,0.191107,0.229753,0.831791
20,8,0.354246,0.459506,0.770928


**Вывод**

В результате исследования можно сделать вывод, что чем меньше детей, тем больше вероятность просрочки по кредиту, за исключением Клиентов с 3 детьми, они занимают 2 место по просрочкам платежей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [21]:
#сгруппируем таблицу по семейному положению и выведем информацию по просроченной задолженности
data_groupby_family_status = data.groupby('family_status').agg({'debt':'sum','family_status':'count'})
#выведем информацию о доле задолженности по каждой категории
data_groupby_family_status['debt_percent'] = data_groupby_family_status['debt']/data['debt'].sum()*100
#узнаем долю каждой категории 
data_groupby_family_status['family_status'] = data_groupby_family_status['family_status']/data['family_status'].count()*100
#определим зависимость количества детей и задолженности
data_groupby_family_status['bound'] = data_groupby_family_status['family_status']/data_groupby_family_status['family_status'] 

#выведем таблицу на экран
display(data_groupby_family_status.sort_values(by = 'bound',ascending = False))


,debt,family_status,debt_percent,bound
family_status,,,,
Не женат / не замужем,274,13.097791,15.738082,1.0
в разводе,85,5.570057,4.882252,1.0
вдовец / вдова,63,4.470029,3.618610,1.0
гражданский брак,388,19.348373,22.286043,1.0
женат / замужем,931,57.513750,53.475014,1.0


**Вывод**

Семейное положение никак не влияет на возврат кредита в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [22]:
#сгруппируем таблицу по семейному положению и выведем информацию по просроченной задолженности
data_groupby_total_income_category = data.groupby('total_income_category').agg({'debt':'sum','total_income_category':'count'})
#выведем информацию о доле задолженности по каждой категории
data_groupby_total_income_category['debt_percent'] = data_groupby_total_income_category['debt']/data['debt'].sum()*100
#узнаем долю каждой категории 
data_groupby_total_income_category['total_income_category'] = data_groupby_total_income_category['total_income_category']/data['total_income_category'].count()*100
#определим зависимость количества детей и задолженности
data_groupby_total_income_category['bound'] = data_groupby_total_income_category['total_income_category']/data_groupby_total_income_category['total_income_category'] 

#выведем таблицу на экран
display(data_groupby_total_income_category.sort_values(by = 'bound',ascending = False))

,debt,total_income_category,debt_percent,bound
total_income_category,,,,
0,23,1.733942,1.321080,1.0
1,331,19.068705,19.012062,1.0
2,661,36.389484,37.966686,1.0
3,368,19.194556,21.137277,1.0
4,358,23.613312,20.562895,1.0


**Вывод**

Очень неожиданно, но уровень дохода не повлиял на возврат кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [23]:
#сгруппируем таблицу по семейному положению и выведем информацию по просроченной задолженности
data_groupby_purpose_category = data.groupby('purpose_category').agg({'debt':'sum','purpose_category':'count'})
#выведем информацию о доле задолженности по каждой категории
data_groupby_purpose_category['debt_percent'] = data_groupby_purpose_category['debt']/data['debt'].sum()*100
#узнаем долю каждой категории 
data_groupby_purpose_category['purpose_category'] = data_groupby_purpose_category['purpose_category']/data['purpose_category'].count()*100
#определим зависимость количества детей и задолженности
data_groupby_purpose_category['bound'] = data_groupby_purpose_category['purpose_category']/data_groupby_purpose_category['purpose_category'] 

#выведем таблицу на экран
display(data_groupby_purpose_category.sort_values(by = 'bound',ascending = False))


,debt,purpose_category,debt_percent,bound
purpose_category,,,,
автомобиль,403,20.070849,23.147616,1.0
недвижимость,782,50.391535,44.916715,1.0
образование,370,18.705137,21.252154,1.0
свадьба,186,10.832479,10.683515,1.0


**Вывод**
Как не странно, но цель получения кредита тоже не имеет большого влияния на возврат кредита в срок.

## Общий вывод
<a id="conclusion_id"></a>

В результате исследования надежности заёмщиков было проверено влияние на возврат кредита в срок таких факторов как:
- количество детей,
- семейное положение,
- цель получения кредита,
- уровень дохода заёмщиков.
Из 4 гипотез явно подтвердилась только 1.
Чем больше у заёмщика детей, тем меньше просрочек по кредиту(см. ниже таблицу корреляции между количеством детей и возвратом кредита в срок).
Влияние остальных изученных критериев если и есть, то незначительное.

Таким образом можно предположить, что возврат кредита в срок в основном зависит от степени ответственности, которая увеличивается при появлении детей и не зависит от уровня дохода,семейного положения и цели получения кредита.



In [63]:
display(
    data_groupby_children
    .reset_index(drop=True)
    .sort_values(by='children',ascending = False)
    .loc[:,['children','bound']]
)

,children,bound
0,65.680060,1.075720
1,22.629813,0.885360
2,9.564650,0.858353
3,1.538175,0.991838
6,0.354246,0.770928
4,0.191107,0.831791
5,0.041950,inf


## Чек-лист готовности проекта
<a id="list_id"></a>

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.